In [10]:
import json
import matplotlib.pyplot as plt
import pandas as pd
import glob
from collections import defaultdict

## Core components

#### Parse JSON logs

In [44]:
def parse_champions(logs_file_iter):
    result = {}
    current_line = next(logs_file_iter, '{}')
    line_json = json.loads(current_line)
    while line_json.get('type', None) == 'ChampionSpawnedReport':
        name = line_json['value']['controller_name']
        coords = line_json['value']['coords']
        champion_data = {
            'name': name,
            'start_coords': coords,
            'last_coords': coords,
            'failed_teps': [],  # every item is a dict {'episode', 'error_text', 'coord'}
            'actions': [],
            'episode_died': 0
        }
        result[name] = champion_data
        current_line = next(logs_file_iter, '{}')
        line_json = json.loads(current_line)
    return result, current_line if current_line != '{}' else None

In [43]:
def parse_game_course(current_line, logs_file_iter, game_report):
    if current_line is None:
        return None
    champions_info = game_report['bots']
    
    line_json = json.loads(current_line)
    while line_json.get('type', None) not in ('GameStartReport', None):
        current_line = next(logs_file_iter, '{}')
        line_json = json.loads(current_line)
    return current_line if current_line != '{}' else None

In [48]:
def parse_games_json_logs(logs_path):
    with open(logs_path) as logs_file:
        logs_file_iter = iter(logs_file)
        result = []
        current_line = next(logs_file_iter, None)
        while current_line is not None:
            game_json = json.loads(current_line)
            arena_json = json.loads(next(logs_file_iter, 'null'))
            menhir_json = json.loads(next(logs_file_iter, 'null'))
            champions, current_line = parse_champions(logs_file_iter)
            if not all((arena_json, menhir_json, champions)):
                print('Not all metadata provided')
                break
            game_report = {
                'game_number': int(game_json['value']['game_number']),
                'mist_episode': -1,
                'menhir_pos': menhir_json['value']['position'],
                'arena_name': arena_json['value']['arena_name'],
                'bots': champions,
                'episodes_number': 0
            }

            current_line = parse_game_course(current_line, logs_file_iter, game_report)
            result.append(game_report)
    return result

#### Parsed data analysis

## Before running the parser
> The parser `.ipynb` is put under `GUPB/log-parse` and all paths to logs are relative to this location. Be sure that the files are downloaded and placed there, because they are not under Git

## Example log

In [49]:
example_path = 'example_log.json'
example_parsed_data = parse_games_json_logs(example_path)

In [51]:
example_parsed_data

[{'game_number': 1,
  'mist_episode': -1,
  'menhir_pos': [9, 9],
  'arena_name': 'isolated_shrine',
  'bots': {'RandomControllerCecilia': {'name': 'RandomControllerCecilia',
    'start_coords': [10, 17],
    'last_coords': [10, 17],
    'failed_teps': [],
    'actions': [],
    'episode_died': 0},
   'RandomControllerBob': {'name': 'RandomControllerBob',
    'start_coords': [11, 14],
    'last_coords': [11, 14],
    'failed_teps': [],
    'actions': [],
    'episode_died': 0},
   'EvaderController--Marwin': {'name': 'EvaderController--Marwin',
    'start_coords': [6, 1],
    'last_coords': [6, 1],
    'failed_teps': [],
    'actions': [],
    'episode_died': 0},
   'RandomControllerDarius': {'name': 'RandomControllerDarius',
    'start_coords': [15, 5],
    'last_coords': [15, 5],
    'failed_teps': [],
    'actions': [],
    'episode_died': 0},
   'RandomControllerAlice': {'name': 'RandomControllerAlice',
    'start_coords': [3, 12],
    'last_coords': [3, 12],
    'failed_teps': [],

## 11:15 games

In [9]:
DIR_PATH_11_15 = '11_15__2021_10_24'
log_files = glob.glob(f'{DIR_PATH_11_15}/*.json')

In [21]:
json_log_path = log_files[0]

In [ ]:
games = 